<a href="https://colab.research.google.com/github/Syamchand123/Lung_cancer_detection/blob/main/lung_cancer_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
import shutil
import os

# Paths to train, test, val directories
dirs = [
    '/content/drive/MyDrive/chest_xray/images/test/labels',
    '/content/drive/MyDrive/chest_xray/images/train/labels',
    '/content/drive/MyDrive/chest_xray/images/val/labels'
]

# Remove labels directories
for dir_path in dirs: # Fixed indentation - 4 spaces
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
        print(f"Removed: {dir_path}")
    else:
        print(f"Not found: {dir_path}")

Not found: /content/drive/MyDrive/chest_xray/images/test/labels
Not found: /content/drive/MyDrive/chest_xray/images/train/labels
Not found: /content/drive/MyDrive/chest_xray/images/val/labels


In [ ]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_data = datagen.flow_from_directory(
    '/content/drive/MyDrive/chest_xray/images/train',
    target_size=(224, 224),  # MobileNetV2 input size
    batch_size=32,
    class_mode='binary'
)

val_data = datagen.flow_from_directory(
    '/content/drive/MyDrive/chest_xray/images/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_data = datagen.flow_from_directory(
    '/content/drive/MyDrive/chest_xray/images/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
# Load the base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # Corrected indentation

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5,  # Keep it low for quick results
    steps_per_epoch=len(train_data),
    validation_steps=len(val_data)
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
 30/163 ━━━━━━━━━━━━━━━━━━━━ 54:21 25s/step - accuracy: 0.7351 - loss: 0.5731

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, precision_recall_curve, auc

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Get true labels and predictions
y_true = test_data.classes  # True labels
y_pred_probs = model.predict(test_data)  # Predicted probabilities
y_pred = np.where(y_pred_probs.ravel() > 0.5, 1, 0)  # Convert probabilities to binary predictions

# Classification report (Precision, Recall, F1-score)
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=test_data.class_indices.keys()))

# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=test_data.class_indices.keys()).plot(cmap="Blues")
plt.title("Confusion Matrix")
plt.show()

# Precision-Recall curve
precision, recall, thresholds = precision_recall_curve(y_true, y_pred_probs.ravel())
pr_auc = auc(recall, precision)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f"PR AUC = {pr_auc:.2f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend(loc="lower left")
plt.grid()
plt.show()


NameError: name 'model' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, precision_recall_curve, auc
import os
from tensorflow.keras.preprocessing.image import array_to_img

# Create a directory for saving results
results_dir = '/content/drive/MyDrive/chest_xray/results'
os.makedirs(results_dir, exist_ok=True)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Get true labels and predictions
y_true = test_data.classes  # True labels
y_pred_probs = model.predict(test_data)  # Predicted probabilities
y_pred = np.where(y_pred_probs.ravel() > 0.5, 1, 0)  # Binary predictions

# Rename classes for Lung Cancer detection
class_names = ['Normal', 'Lung Cancer']

# Classification report
print("\nClassification Report:")
class_report = classification_report(y_true, y_pred, target_names=class_names)
print(class_report)

# Save the classification report to a file
with open(os.path.join(results_dir, 'classification_report.txt'), 'w') as f:
    f.write(class_report)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=class_names)

# Plot and save Confusion Matrix
plt.figure(figsize=(8, 6))
disp.plot(cmap=plt.cm.Blues, values_format='d')
plt.title("Confusion Matrix")
plt.savefig(os.path.join(results_dir, 'confusion_matrix.png'))
plt.show()

# Precision-Recall Curve
precision, recall, thresholds = precision_recall_curve(y_true, y_pred_probs)
pr_auc = auc(recall, precision)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.', label=f'PR Curve (AUC = {pr_auc:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.grid()
plt.savefig(os.path.join(results_dir, 'precision_recall_curve.png'))
plt.show()

# F1 Curve
f1_scores = 2 * (precision * recall) / (precision + recall)
plt.figure(figsize=(8, 6))
plt.plot(thresholds, f1_scores[:-1], marker='.', label='F1 Score')
plt.xlabel('Threshold')
plt.ylabel('F1 Score')
plt.title('F1 Curve')
plt.legend()
plt.grid()
plt.savefig(os.path.join(results_dir, 'f1_curve.png'))
plt.show()

# Sample detected images (Random 5 from test set)
print("\nSample Detection Results:")
sample_indices = np.random.choice(len(test_data), 5, replace=False)

for i, idx in enumerate(sample_indices):
    img, label = test_data[idx]  # Fetch the image and label
    pred_label = 'Lung Cancer' if y_pred[idx] == 1 else 'Normal'
    true_label = 'Lung Cancer' if y_true[idx] == 1 else 'Normal'

    plt.imshow(array_to_img(img[0]))
    plt.title(f"True: {true_label}, Predicted: {pred_label}")
    plt.axis('off')
    plt.savefig(os.path.join(results_dir, f'sample_{i + 1}.png'))
    plt.show()

print(f"Results saved to {results_dir}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve

# Get predictions and ground truth
y_pred_prob = model.predict(test_data)  # Predicted probabilities
y_true = test_data.classes              # Ground truth labels

# Compute Precision and Recall values
precision, recall, thresholds = precision_recall_curve(y_true, y_pred_prob)

# Plot Precision Curve (P Curve)
plt.figure(figsize=(8, 6))
plt.plot(thresholds, precision[:-1], label="Precision Curve", color="blue")
plt.xlabel("Thresholds")
plt.ylabel("Precision")
plt.title("Precision Curve")
plt.legend()
plt.grid()
plt.show()

# Plot Recall Curve (R Curve)
plt.figure(figsize=(8, 6))
plt.plot(thresholds, recall[:-1], label="Recall Curve", color="green")
plt.xlabel("Thresholds")
plt.ylabel("Recall")
plt.title("Recall Curve")
plt.legend()
plt.grid()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Get predictions
y_pred_prob = model.predict(test_data)  # Predicted probabilities
y_pred_classes = np.where(y_pred_prob > 0.5, 1, 0)  # Predicted classes (binary classification)

# Ground truth labels
y_true = test_data.classes

# Get filenames
filenames = test_data.filepaths

# Find correctly predicted indices
correct_indices = np.where(y_pred_classes == y_true)[0]

# Display a few correctly detected images
num_images = 5  # Number of images to display
plt.figure(figsize=(15, 10))

for i, idx in enumerate(correct_indices[:num_images]):
    img = plt.imread(filenames[idx])
    plt.subplot(1, num_images, i + 1)
    plt.imshow(img)
    plt.title(f"Correctly Detected\nLabel: {y_true[idx]}")
    plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Get predictions
y_pred_prob = model.predict(test_data)  # Predicted probabilities
y_pred_classes = np.where(y_pred_prob > 0.5, 1, 0)  # Predicted classes (binary classification)

# Ground truth labels
y_true = test_data.classes

# Get filenames
filenames = test_data.filepaths

# Find correctly predicted indices
correct_indices = np.where(y_pred_classes == y_true)[0]

# Display a few unique correctly detected images
num_images = 5  # Number of unique images to display
plt.figure(figsize=(15, 10))

unique_indices = set()  # To store unique indices
count = 0  # Track displayed images

for idx in correct_indices:
    if idx not in unique_indices:  # Avoid duplicates
        unique_indices.add(idx)
        img = plt.imread(filenames[idx])
        plt.subplot(1, num_images, count + 1)
        plt.imshow(img)
        plt.title(f"Correctly Detected\nLabel: {y_true[idx]}")
        plt.axis("off")
        count += 1
        if count == num_images:  # Stop after displaying the desired number
            break

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Map class indices to labels
class_labels = {0: "Normal", 1: "Abnormal"}

# Get predictions
y_pred_prob = model.predict(test_data)  # Predicted probabilities
y_pred_classes = np.where(y_pred_prob > 0.5, 1, 0)  # Predicted classes (binary classification)

# Ground truth labels
y_true = test_data.classes

# Get filenames
filenames = test_data.filepaths

# Find correctly predicted indices
correct_indices = np.where(y_pred_classes == y_true)[0]

# Display a few unique correctly detected images
num_images = 5  # Number of unique images to display
plt.figure(figsize=(15, 10))

unique_indices = set()  # To store unique indices
count = 0  # Track displayed images

for idx in correct_indices:
    if idx not in unique_indices:  # Avoid duplicates
        unique_indices.add(idx)
        img = plt.imread(filenames[idx])

        # True label and predicted label
        true_label = class_labels[y_true[idx]]
        predicted_label = class_labels[y_pred_classes[idx]]

        # Display image
        plt.subplot(1, num_images, count + 1)
        plt.imshow(img)
        plt.title(f"True: {true_label}\nPredicted: {predicted_label}")
        plt.axis("off")
        count += 1
        if count == num_images:  # Stop after displaying the desired number
            break

plt.tight_layout()
plt.show()
